In [1]:
import sys
sys.path.insert(0, '../src/')
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import scipy.sparse as sp
from scipy.sparse import load_npz
import graph_statistics
import utils

from evaluation import (Evaluation, tabular_from_statistics, df_from_tabular, compute_original_statistics,
                        boxplot, df_from_dataset, errorbar_plot)

#### True graph

In [2]:
_A_obs = load_npz('../data/datasets/citeseer.npz')

val_share = 0.1
test_share = 0.05
seed = 481516234

train_ones, val_ones, val_zeros, test_ones, test_zeros = utils.train_val_test_split_adjacency(_A_obs, val_share, test_share, seed, undirected=True, connected=True, asserts=False)

train_graph = sp.csr_matrix((np.ones(len(train_ones)),(train_ones[:,0], train_ones[:,1])))
assert (train_graph.toarray() == train_graph.toarray().T).all()

#### Statistic Functions

In [10]:
statistic_fns = {#'Assortativity':graph_statistics.assortativity,
                 #'Average Degree':graph_statistics.average_degree,
                 #'Claw Count':graph_statistics.claw_count,
#                  'Clustering Coefficient':graph_statistics.clustering_coefficient,
                 #'Characteristic Path Length':graph_statistics.compute_cpl,
#                  'Edge Distribution Entropy':graph_statistics.edge_distribution_entropy,
                 #'Gini':graph_statistics.gini,
#                   'Intra com.' : intra_community_density,
#                   'Inter com.' : inter_community_density,
                 #'LCC Size':graph_statistics.LCC,
                 'Max Degree':graph_statistics.max_degree,
                 #'Min Degree':graph_statistics.min_degree,
                 #'Num Connected Components':graph_statistics.num_connected_components,
#                  'Power Law α':graph_statistics.power_law_alpha,
#                  'Spectral Gap':graph_statistics.spectral_gap,
#                  'Square Count':graph_statistics.square_count,
#                  'Triangle Count':graph_statistics.triangle_count,
#                  'Wedge Count':graph_statistics.wedge_count,
                 }

#### Evaluate 

In [11]:
df, evals = df_from_dataset(path_to_dataset='../logs/test/',
                            statistic_fns=statistic_fns,
                            target_overlap=0.55,
                            original_graph=train_graph,
                            max_trials=None)
df

,Max Degree,Edge Overlap (%),ROC-AUC Score,Average Precision,Time (s)
test,72.0,1.00000,NaN,NaN,NaN
baseline_NetGan,49.4,0.58351,0.936438,0.937052,3070.237604


#### Plot at selected statistics

In [9]:
df_for_paper = df.reindex(['citeseer', 'baseline_conf', 'baseline_FA', 'baseline_FT', 'baseline_FS', 
                           'baseline_FM',
                           'baseline_transition_ce_loss',
                           'baseline_Netgan_sample_many_eo_52', 
                           'Ours'])
# df_for_paper = df.reindex(['CORA-ML', 'baseline_conf', 'baseline_FS'])
df_for_paper[['Max Degree', 'Assortativity', 'Triangle Count', 'Square Count', 'Power Law α', 'Clustering Coefficient',
              'Spectral Gap', 'ROC-AUC Score', 'Time (s)', 'Edge Overlap (%)']]

,Max Degree,Assortativity,Triangle Count,Square Count,Power Law α,Clustering Coefficient,Spectral Gap,ROC-AUC Score,Time (s),Edge Overlap (%)
citeseer,72.00,-0.014617,483.00,1866.0,2.237941,0.012596,0.001633,NaN,NaN,1.000000
baseline_conf,70.25,-0.013547,108.35,282.8,2.239297,0.002969,0.018775,NaN,0.078739,0.561614
baseline_FA,34.40,-0.004751,88.60,188.0,2.094662,0.012370,0.006842,0.607824,11.604076,0.571896
baseline_FT,35.60,-0.022396,119.00,364.2,2.149519,0.014106,0.003990,0.824876,8.178115,0.571062
baseline_FS,48.00,0.019383,108.40,161.2,2.176689,0.006386,0.004104,0.361915,12.084771,0.569971
baseline_FM,32.00,0.000533,87.40,162.0,2.086574,0.014204,0.010269,0.622608,108.311107,0.561630
baseline_transition_ce_loss,43.80,-0.093193,126.20,477.2,2.131933,0.009992,0.016985,0.906497,42.142151,0.524158
baseline_Netgan_sample_many_eo_52,52.00,-0.074473,360.80,478.2,2.151062,0.020301,0.007442,0.951474,1584.728088,0.565287
Ours,44.00,-0.092912,106.00,317.6,2.173523,0.007871,0.015597,0.858446,4.470461,0.555149
